# 2. Scene Catalog Search (raw, selected, time series)

This notebook queries CDSE STAC for Sentinel‑2 scenes intersecting the Thessaloniki AOI, then builds:

- `scenes_catalog.csv` (raw STAC hits)
- `scenes_selected.csv` (best scenes selected by the TileSelector rules)
- `time_serie.csv` (anchor_date → chosen acquisition + tile ids)

All outputs are written under `outputs/tables/` via `RepoPaths`.


## A. Imports

(If you run this from a fresh environment, install dependencies first.)

In [ ]:
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt

from thess_geo_analytics.pipelines.BuildSceneCatalogPipeline import (
    BuildSceneCatalogPipeline,
    BuildSceneCatalogParams,
)
from thess_geo_analytics.utils.RepoPaths import RepoPaths


## B. Configure the search

Adjust the parameters below to trade off *coverage, quality, and performance*.

Suggested **dev-friendly** settings:
- 365–730 days
- `cloud_cover_max=30`
- `max_items=1000`
- `n_anchors=12` (monthly-ish)
- `window_days=21`


In [ ]:
# --- Inputs ---
aoi_path = Path("aoi/EL522_Thessaloniki.geojson")

# --- Search horizon ---
# Option 1: relative (days back from today)
days = 365  # try 365 for dev; increase to 2190 for ~6y once stable

# --- STAC filters ---
cloud_cover_max = 30.0
max_items = 1000
collection = "sentinel-2-l2a"

# --- Selection / time-series controls ---
use_tile_selector = True
full_cover_threshold = 0.999
allow_union = True
max_union_tiles = 2

# Regular time grid (anchors) + window around each anchor
n_anchors = 12      # e.g., 12 points across the period
window_days = 21    # +/- 10 days (approx)


## C. Run the pipeline

This will create/overwrite the three CSV files in `outputs/tables/`.

In [ ]:
pipe = BuildSceneCatalogPipeline(aoi_path=aoi_path)

params = BuildSceneCatalogParams(
    days=days,
    cloud_cover_max=cloud_cover_max,
    max_items=max_items,
    collection=collection,
    use_tile_selector=use_tile_selector,
    full_cover_threshold=full_cover_threshold,
    allow_union=allow_union,          # name in the new selector-based pipeline
    max_union_tiles=max_union_tiles,  # name in the new selector-based pipeline
    n_anchors=n_anchors,
    window_days=window_days,
)

out = pipe.run(params)
print("Pipeline returned:", out)

raw_csv = RepoPaths.table("scenes_catalog.csv")
sel_csv = RepoPaths.table("scenes_selected.csv")
ts_csv  = RepoPaths.table("time_serie.csv")

print("Raw:", raw_csv)
print("Selected:", sel_csv)
print("Time series:", ts_csv)


## D. Load outputs

In [ ]:
raw_df = pd.read_csv(raw_csv, parse_dates=["datetime"])
sel_df = pd.read_csv(sel_csv, parse_dates=["datetime"]) if sel_csv.exists() else pd.DataFrame()
ts_df  = pd.read_csv(ts_csv, parse_dates=["anchor_date","acq_dt"]) if ts_csv.exists() else pd.DataFrame()

print("raw_df:", raw_df.shape)
print("sel_df:", sel_df.shape)
print("ts_df:", ts_df.shape)


### Quick peek

In [ ]:
display(raw_df.head(5))
if not sel_df.empty:
    display(sel_df.head(5))
if not ts_df.empty:
    display(ts_df.head(10))


## E. Sanity checks

- How many unique acquisition dates did we get?
- Are we reusing the same acquisition for multiple anchors (expected if the catalog is sparse or the window is large)?


In [ ]:
if not raw_df.empty:
    print("Raw unique acquisition dates:", raw_df["datetime"].dt.date.nunique())

if not sel_df.empty:
    print("Selected unique acquisition dates:", sel_df["datetime"].dt.date.nunique())

if not ts_df.empty:
    print("Anchors:", ts_df["anchor_date"].nunique())
    print("Unique acquisitions used:", ts_df["acq_dt"].dt.date.nunique())
    print("Most reused acquisitions (top 5):")
    display(
        ts_df.assign(acq_date=ts_df["acq_dt"].dt.date)
            .groupby("acq_date")
            .size()
            .sort_values(ascending=False)
            .head(5)
            .rename("anchors_using_this_acq")
            .reset_index()
    )


## F. Plot: cloud over anchor dates

If your `time_serie.csv` includes `cloud_score` (recommended), this shows the quality along the regular time grid.


In [ ]:
if not ts_df.empty and "cloud_score" in ts_df.columns:
    ts_df = ts_df.sort_values("anchor_date")
    plt.figure()
    plt.plot(ts_df["anchor_date"], ts_df["cloud_score"], marker="o")
    plt.xlabel("anchor_date")
    plt.ylabel("cloud_score (max over union)")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.show()
else:
    print("No time series (or missing cloud_score column) to plot.")


## G. Next steps

- Increase `days` and `max_items` gradually once you stop seeing API errors.
- If the AOI is not fully covered by a single tile, keep `max_union_tiles=2`.
- If coverage is consistently low (`coverage_frac`), verify the AOI geometry and the STAC footprints, and/or allow unions.
